**🚚 Business Case: Retail Inventory Allocation**
 
 A company has two warehouses (W1 and W2) and three retail stores (S1, S2, S3). There are two products (P1 and P2). Each warehouse has limited stock, and each store has demand. Shipping costs from each warehouse to each store (per unit) are known. We want to minimize total shipping cost while meeting all store demands and not exceeding warehouse stocks.

In [8]:
import pulp

In [6]:
warehouses = ["W1", "W2"]
stores = ["S1", "S2", "S3"]
products = ["P1", "P2"]

# Stock at warehouses
stock = {("W1","P1"):100, ("W1","P2"):80,
         ("W2","P1"):90,  ("W2","P2"):120}

# Demand at stores
demand = {("S1","P1"):70, ("S1","P2"):60,
          ("S2","P1"):80, ("S2","P2"):70,
          ("S3","P1"):40, ("S3","P2"):50}

# Shipping cost per unit
cost = {("W1","S1","P1"):4, ("W1","S2","P1"):5, ("W1","S3","P1"):6,
        ("W2","S1","P1"):5, ("W2","S2","P1"):4, ("W2","S3","P1"):3,
        ("W1","S1","P2"):6, ("W1","S2","P2"):5, ("W1","S3","P2"):4,
        ("W2","S1","P2"):4, ("W2","S2","P2"):6, ("W2","S3","P2"):5}

In [7]:
# 1️ Define problem
prob = pulp.LpProblem("Warehouse_to_Store_Allocation", pulp.LpMinimize)

# 2️ Decision variables: ship[(w,s,p)]
ship = pulp.LpVariable.dicts(
    "ship", (warehouses, stores, products),
    lowBound=0, cat="Integer"
)

# 3️ Objective: minimize shipping cost
prob += pulp.lpSum(
    cost[(w,s,p)] * ship[w][s][p]
    for w in warehouses for s in stores for p in products
), "Total_Shipping_Cost"

# 4️ Constraints:
# a) Don't exceed warehouse stock
for w in warehouses:
    for p in products:
        prob += pulp.lpSum(ship[w][s][p] for s in stores) <= stock[(w,p)], f"Stock_{w}_{p}"

# b) Meet store demand
for s in stores:
    for p in products:
        prob += pulp.lpSum(ship[w][s][p] for w in warehouses) >= demand[(s,p)], f"Demand_{s}_{p}"

In [3]:
# 5️ Solve the model
prob.solve()
print("Status:", pulp.LpStatus[prob.status])


Status: Optimal


In [4]:
# Shipping plan
for w in warehouses:
    for s in stores:
        for p in products:
            qty = ship[w][s][p].varValue
            if qty > 0:
                print(f"Ship {qty} units of {p} from {w} to {s}")

print("Total shipping cost ₹", pulp.value(prob.objective))


Ship 70.0 units of P1 from W1 to S1
Ship 30.0 units of P1 from W1 to S2
Ship 70.0 units of P2 from W1 to S2
Ship 10.0 units of P2 from W1 to S3
Ship 60.0 units of P2 from W2 to S1
Ship 50.0 units of P1 from W2 to S2
Ship 40.0 units of P1 from W2 to S3
Ship 40.0 units of P2 from W2 to S3
Total shipping cost ₹ 1580.0
